In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import scipy.stats as stats
from scipy.interpolate import CubicSpline

from sklearn.model_selection import train_test_split

In [2]:
X = stats.uniform(-5, 5).rvs(20000)
epsilon = np.random.normal(0, 0.3, 20000)
y = np.sin(X) + np.cos(np.multiply(X, 2)) + np.multiply(np.sin(np.multiply(X, 3)), np.exp(-X)) + epsilon
df = pd.DataFrame({'X': X, 'y': y})
df.head()

,X,y
0,-3.446108,26.116616
1,-2.681290,-14.274645
2,-1.990206,0.565687
3,-2.969351,-8.841178
4,-1.101340,-1.562579


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.999, random_state=42)
X_train

array([-3.62795113, -1.39486665, -0.52178919, -4.47019516, -2.10545542,
       -3.11983887, -1.00473074, -3.9030946 , -1.4319406 , -0.23795808,
       -4.44346681, -0.89758708, -2.96963657, -0.76562626, -1.82406809,
       -0.56995595, -3.29386878, -2.21875958, -1.41144694, -0.14664849])

In [4]:
px.scatter(x=X_train, y=y_train, title='Training Data')

In [5]:
from scipy.interpolate import CubicSpline
from scipy.linalg import lstsq
import numpy as np

def fit_spline(X_train, y_train, df):
    n = len(X_train)
    sort_idx = np.argsort(X_train)
    X_train = X_train[sort_idx]
    y_train = y_train[sort_idx]

    if df <= n:
        cs = CubicSpline(X_train[:df], y_train[:df], bc_type='natural')
        return cs
    else:
        # Constructing the spline basis matrix
        knots = np.linspace(X_train[0], X_train[-1], df-2)
        basis_matrix = np.column_stack([X_train**3, X_train**2, X_train, np.ones_like(X_train)] +
                                       [np.piecewise(X_train, [X_train < k, X_train >= k], [0, lambda x: (x - k)**3]) for k in knots])
        
        # Calculating minimum norm solution
        coef, _, _, _ = lstsq(basis_matrix, y_train)

        # Constructing the spline object from the coefficients
        cs = CubicSpline(X_train, np.dot(basis_matrix, coef), bc_type='natural')
        return cs

In [6]:
splines = {f"{df}":fit_spline(X_train, y_train, df) for df in range(4, 51)}

In [7]:
from sklearn.metrics import mean_squared_error as mse
test_error = {k:mse(splines[k](X_test), y_test) for k in splines.keys()}

In [10]:
df = pd.DataFrame({'df': list(test_error.keys()), 'test_error': list(test_error.values())})
df['df'] = df['df'].astype(int)
df['test_error'] = df['test_error'].astype(float).round(2)
df

,df,test_error
0,4,36131943.07
1,5,1399405.91
2,6,619778.40
3,7,3035323.38
4,8,8410.12
5,9,942.45
6,10,7983.53
7,11,223.30
8,12,20613.72
9,13,3946969.50


In [9]:
px.line(df, x='df', y='test_error', title='Test error vs. df')

In [54]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, ElasticNetCV, RidgeCV
from sklearn.metrics import mean_squared_error
import numpy as np
import plotly.express as px

In [55]:
# Generate data
np.random.seed(42)
X = np.random.uniform(-5, 5, 10000)
y = np.sin(X) + np.cos(2*X) + np.power(np.sin(X)*np.exp(-X), -1) + np.random.normal(0, 0.3, 10000) # Change this to your desired f(X)

In [56]:
# Split data
X_train, X_test = X[:20].reshape(-1, 1), X[20:].reshape(-1, 1)
y_train, y_test = y[:20], y[20:]

In [57]:
RidgeCV?

Init signature:
RidgeCV(
    alphas=(0.1, 1.0, 10.0),
    *,
    fit_intercept=True,
    scoring=None,
    cv=None,
    gcv_mode=None,
    store_cv_values=False,
    alpha_per_target=False,
)
Docstring:     
Ridge regression with built-in cross-validation.

See glossary entry for :term:`cross-validation estimator`.

By default, it performs efficient Leave-One-Out Cross-Validation.

Read more in the :ref:`User Guide <ridge_regression>`.

Parameters
----------
alphas : array-like of shape (n_alphas,), default=(0.1, 1.0, 10.0)
    Array of alpha values to try.
    Regularization strength; must be a positive float. Regularization
    improves the conditioning of the problem and reduces the variance of
    the estimates. Larger values specify stronger regularization.
    Alpha corresponds to ``1 / (2C)`` in other linear models such as
    :class:`~sklearn.linear_model.LogisticRegression` or
    :class:`~sklearn.svm.LinearSVC`.
    If using Leave-One-Out cross-validation, alphas must be posi

In [63]:
test_errors = []
alphas = np.linspace(2, 4, 100)
for degree in range(1, 51):
    # Transform to polynomial features
    poly = PolynomialFeatures(degree=degree)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.transform(X_test)

    # Scale data
    scaler = StandardScaler()
    X_train_poly = scaler.fit_transform(X_train_poly)
    X_test_poly = scaler.transform(X_test_poly)
    
    # Fit the model

    model = RidgeCV(cv=5, alphas=alphas, scoring='neg_mean_squared_error')
    model.fit(X_train_poly, y_train)
    print(f"Degree: {degree}, Alpha: {model.alpha_:.3f}, MSE: {model.best_score_:.3f}")
    alphas = np.linspace(model.alpha_ - 1, model.alpha_ + 1, 100)

    
    # Predict and calculate test error
    y_pred = model.predict(X_test_poly)
    mse = mean_squared_error(y_test, y_pred)
    test_errors.append(mse)

Degree: 1, Alpha: 3.273, MSE: -1442.238
Degree: 2, Alpha: 2.313, MSE: -932.308
Degree: 3, Alpha: 3.313, MSE: -870.382
Degree: 4, Alpha: 4.313, MSE: -907.913
Degree: 5, Alpha: 5.313, MSE: -939.497
Degree: 6, Alpha: 6.313, MSE: -961.933
Degree: 7, Alpha: 7.212, MSE: -990.029
Degree: 8, Alpha: 6.212, MSE: -1001.191
Degree: 9, Alpha: 5.212, MSE: -1013.068
Degree: 10, Alpha: 4.212, MSE: -1012.017
Degree: 11, Alpha: 3.212, MSE: -1007.143
Degree: 12, Alpha: 2.919, MSE: -1012.204
Degree: 13, Alpha: 2.768, MSE: -1008.857
Degree: 14, Alpha: 3.323, MSE: -1017.581
Degree: 15, Alpha: 3.313, MSE: -1015.924
Degree: 16, Alpha: 3.808, MSE: -1024.422
Degree: 17, Alpha: 3.859, MSE: -1023.891
Degree: 18, Alpha: 4.313, MSE: -1031.779
Degree: 19, Alpha: 4.404, MSE: -1031.898
Degree: 20, Alpha: 4.798, MSE: -1039.012
Degree: 21, Alpha: 4.909, MSE: -1039.468
Degree: 22, Alpha: 5.283, MSE: -1045.769
Degree: 23, Alpha: 5.374, MSE: -1046.374
Degree: 24, Alpha: 5.727, MSE: -1051.892
Degree: 25, Alpha: 5.818, MSE: 

In [64]:
# Plot the results
px.line(x=range(1, 51), y=test_errors, title='Test error vs. Polynomial degree', labels={'x': 'Polynomial degree', 'y': 'Test error'}, log_y=True)